# Trabajo final PLN

Análisis de sentimiento

In [1]:
#!pip install Flask-Bootstrap
#!pip install textblob

## 1. Recolección de datos

Para la recolección de datos se usó la API tweepy, estra api permite interactuar con twitter para la publicación y recolección de tweets, en nuestro caso lo usaremos para extraer datos de publicaciones por fechas a partir de un topic.

Importación de librerias

In [2]:
import os
import tweepy as tw
import numpy as np
import pandas as pd
import json 

from flask import Flask, render_template,request,url_for,Response
from flask_bootstrap import Bootstrap

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#plt.style.use('seaborn')

from textblob import TextBlob,Word 
import random 
import time
import datetime

# svm
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

import pickle

Credenciales

In [3]:
consumer_key= 'PCAI4xyHdvwVcwfcw1FuWslrM'
consumer_secret= 'PgOnYL2cMqUcoscPR8t9rGEFEc65PhBahVfOYLJxR7V2mwPnyq'
access_token= '3158149931-HkzDndqf0GxdpGt6bVkySL0ZXCevxdeotmEeVbJ'
access_token_secret= 'jL7WnaaCphpVtyavexHRlZCu9fkuZkw6J0T4OBXMRHdER'

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Funcion para acceder por fecha a tweets mediante api tweepy 

In [5]:
def tweet_days(search_words, month, day):

    dt = datetime.datetime.today()
    year = dt.year
    date_since = datetime.datetime(year, month, day, 0, 0, 0)
    date_until =   datetime.datetime(year, month, day+1, 0, 0, 0)

    tweets = tw.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since,
                       until=date_until,
                       result_type="recent",
                       sleep_on_rate_limit=False
                       ).items(25)

    return tweets

Funcion para recopilar tweets en dataframe

In [6]:
def get_tweets_1(search_words):

    df = pd.DataFrame(columns=['date', 'user', 'text'])

    dt = datetime.datetime.today()
    days = dt.day

    for day in range(days-25,days):
        data = []
        if day <= 0:
            day = 30 + day
            month = dt.month - 1
            #print(month, day)
            tweets = tweet_days(search_words, month, day)

        else:
            month = dt.month
            #print(month,day)
            tweets = tweet_days(search_words, month, day)

        data = [[tweet.created_at, tweet.user.screen_name, tweet.text] for tweet in tweets]
        df2 = pd.DataFrame(data = data, columns=['date', 'user', 'text'])
        df = df.append(df2, ignore_index = True)
    
    return df

In [7]:
def get_tweets(search_words):

    df = pd.DataFrame(columns=['date', 'user', 'text'])

    date_since = datetime.datetime(2021, 7, 1, 0, 0, 0)
    #date_until =   datetime.datetime(year, month, day+1, 0, 0, 0)

    tweets = tw.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since,
                       #until=date_until,
                       result_type="recent",
                       sleep_on_rate_limit=False
                       ).items(1000)
    data = [[tweet.created_at, tweet.user.screen_name, tweet.text] for tweet in tweets]
    df = pd.DataFrame(data = data, columns=['date', 'user', 'text'])
    return df

## 2. Preprocesamiento

Se usó la libreria NLTK y re, el preprocesamiento consiste en:

- Cambio de mayusculas a minusculas.
- Eliminación de menciones "@...", retweets "RT @..." y enlaces webs "htps://..." 
- Eliminación de stopwords
- Eliminación de emojis

Importación de librerias para preprocesamiento

In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk import FreqDist
import re

[nltk_data] Downloading package punkt to /home/jh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Función para preprocesamiento de tweets

In [9]:
def text_prep(text:str):
    from nltk.corpus import stopwords
    stopwords = stopwords.words('english')
    
    tokens = [] 
    text = re.sub('https\S+', '', text)
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)

    text = regrex_pattern.sub(r'',text)
    #text = re.sub('Replying to \n@\S+', '', text)
    text = re.sub('RT @\S+', '', text)
    text = re.sub('@\S+', '', text)
    text = re.sub('#\S+', '', text)

    for w in word_tokenize(text):
        w = w.lower()
        if ((re.search('[a-zA-Z]', w)) and (w not in stopwords)): tokens.append(w)
    return ' '.join(tokens)

In [10]:
def get_sentiment(text:str):
    blob = TextBlob(text)
    blob_sentiment = blob.sentiment.polarity
    if blob_sentiment >= 0:
        blob_sentiment = 1
    else:
        blob_sentiment = 0
    return blob_sentiment

In [11]:
def get_sentiment_svm(text:str):
    clf = pickle.load(open('clf.pkl', 'rb'))
    tfidf = pickle.load(open('tfidf.pkl', 'rb'))
    sentiment = clf.predict(tfidf.transform([text]))
    return sentiment[0]

In [12]:
%%writefile templates/index.html
{% extends "bootstrap/base.html" %}

{% block content %}

<style type="text/css">
        body{
    font:15px/1.5 Arial, Helvetica,sans-serif;
}
        .spinner-1:before{
            content: "";
            box-sizing: border-box;
            position: absolute;
            top:50%;
            left: 50%;
            height: 60px;
            width: 60px;
            margin-top: -30px;
            margin-left: -30px;
            border-radius: 50%;
            border:6px solid transparent;
            border-top-color: blue;
            animation: spinner 0.7s linear infinite;
        }
                 .jumbotron text-center{
                  background-color:red;
                  text-color:white;
                   }
        @keyframes spinner {
            to {
                transform: rotate(360deg);
            }

        }
        li { background-color:#BDBDBD; }
        li:nth-child(odd) { background-color:green; }
        </style>


<div class="container">
    <div class="text-center">
        <h3><b>TWITTER SENTIMENT ANALYSIS NLP<b></h3>
    </div>
</div>

<div class="container jumbotron">
    <form method="POST" action="{{ url_for('analyse')}}" id="myForm">
        
    <label ><strong>Enter topic<strong></label><br>
    <textarea class="form-control" rows="1" cols="10" name="rawtext"></textarea><br>

    <input type="submit" onclick="myAnalyser()" value="Submit" class="btn btn-primary ">
    <input type="button" onclick="myFunction()" value="Clear" class="btn btn-outline-dark">

    <a href="{{ url_for('index')}}" type="button" class="btn btn-danger" > Reset</a>
    
    </form>
    
</div>
<br/>

<div class="main">
<div class="container">
    <div class="card">

  <div class="card-body">
    <div class="card-text"> 
        <h5>Topic</h5>
        <p style="color:#0091EA;font-family:sans-serif;">{{ received_text }}</p>
        <hr/>
        <li class="list-group-item list-group-item-info"><span style="color:black">Tweets: {{n_twwets}}</span></li>

        <hr/>
<br/>
    <p>Time Elapsed: <span style="color:#0091EA;">{{ final_time }} </span> seconds to analyse</p>
       
  </div>
  <div class="card-footer text-muted">
    <table class="table table-striped table-dark" >
      <thead>
        <tr>
          <th scope="col">Tweet</th>
          <th scope="col">Polarity</th>
          <th scope="col">Polarity_svm</th>
        </tr>
      </thead>
      <tbody>
        {% for row in data %}
        <tr>
          <td>{{row['text']}}</td>
          <td>{{row['polarity']}}</td>
          <td>{{row['polarity_svm']}}</td>
        {% endfor %}
        </tr>
    </tbody></table>
 
    <div class="container">
            
        <div class="text-center">

            <h4>Wordcloud</h4>

            <img src={{url_wc}} alt="Chart_wc" width="800" >
            
        </div>
            
        <div class="text-center">
            <h4>{{name}}</h4>

            <img src={{url}} alt="Chart" width="700" >
        </div>
    </div>
</div>

    
</div>
</div>


{% endblock %}


<!-- Scripts starts here -->
{% block scripts %}

{{ super() }}

<script>
function myFunction() {
    document.getElementById("myForm").reset();
}
</script>
<script>
function myAnalyser() {
    document.querySelector('.main div').style.display = 'none';
    //Hide the main division
    document.querySelector('.main').classList.add('spinner-1');
    // Server request
    setTimeout(() => {
    document.querySelector('.main').classList.remove('spinner-1');
    //Remove the animation
    document.querySelector('.main div').style.display = 'block';
    //Show the main division
    },15000);//Number of seconds to last
}
</script>

<!-- Prevent it from being overwritten -->

{% endblock %}

Overwriting templates/index.html


In [13]:
app = Flask(__name__)
Bootstrap(app)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/analyse',methods=['POST'])
def analyse():
    summary = ''
    final_time = ''
    start = time.time()
    if request.method == 'POST':
        topic = request.form['rawtext']
        
        #NLP Stuff
        df = get_tweets(topic)
        df['text_prep'] = df.text.apply(text_prep)
        df['polarity'] = df.text.apply(get_sentiment)
        df['polarity_svm'] = df.text.apply(get_sentiment_svm)
        
        df = df.sort_values('date')
        dates = df['date']
        emotion = df['polarity']
        emotion_svm = df['polarity_svm']
        data_tweet = df[['text','polarity','polarity_svm']].tail(10)
        print(data_tweet)
        plt.clf()
        plt.figure(figsize=[8,4])
        plt.plot(dates, emotion, 'o', label = 'emotions')
        plt.plot(dates, emotion_svm, '*', label = 'emotions_svm')
        date_format = mdates.DateFormatter('%m/%d')
        plt.gca().xaxis.set_major_formatter(date_format)
        plt.xticks(rotation = 30)
        plt.ylim([-0.25, 1.25])
        plt.legend(['txtb','svm'])
        plt.grid()
        
        plt.savefig('static/images/new_plot.png')
        
        plt.clf()
        from wordcloud import WordCloud
        wordcloud = WordCloud().generate(' '.join(df.text_prep))

        plt.figure(figsize=[15,8])
        plt.imshow(wordcloud)
        plt.savefig('static/images/cloud_plot.png')
        
        
        #print(df.tail(10))
        
        rawtext = topic
        blob = TextBlob(rawtext)
        received_text2 = blob
        blob_sentiment,blob_subjectivity = blob.sentiment.polarity ,blob.sentiment.subjectivity
        number_of_tokens = len(list(blob.words))
        
        # Extracting Main Points
        nouns = list()
        for word, tag in blob.tags:
            if tag == 'NN':
                nouns.append(word.lemmatize())
                len_of_words = len(nouns)
                rand_words = random.sample(nouns,len(nouns))
                final_word = list()
                for item in rand_words:
                    word = Word(item).pluralize()
                    final_word.append(word)
                    summary = final_word
                    end = time.time()
                    final_time = round(end-start,2)


    return render_template('index.html',received_text = received_text2,n_twwets = len(df),data=data_tweet.to_dict(orient='records'),final_time=final_time, name = 'Sentiment plot', url ='/static/images/new_plot.png', url_wc ='/static/images/cloud_plot.png' )

#if __name__ == '__main__':
#    app.run(debug=True)

In [14]:
import socket

from time import sleep
sleep(1)

import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()

sleep(5)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.100.9:5000/ (Press CTRL+C to quit)
192.168.100.9 - - [20/Aug/2021 18:43:52] "GET / HTTP/1.1" 200 -
192.168.100.9 - - [20/Aug/2021 18:43:52] "GET /alt=%22Chart_wc%22 HTTP/1.1" 404 -
192.168.100.9 - - [20/Aug/2021 18:43:52] "GET /alt=%22Chart%22 HTTP/1.1" 404 -
192.168.100.9 - - [20/Aug/2021 18:44:12] "GET /static/images/new_plot.png HTTP/1.1" 304 -
192.168.100.9 - - [20/Aug/2021 18:44:13] "GET /static/images/new_plot.png HTTP/1.1" 304 -


                                                text  polarity  polarity_svm
9  RT @charliespiering: At 1:00 pm Biden will del...         1             1
8  RT @larryelder: So Biden’s VP is visiting Saig...         1             1
7  RT @CBSNews: WATCH: A small, crying baby was p...         0             1
6  RT @CBSNews: WATCH: A small, crying baby was p...         0             1
5  RT @CrazyTrain88217: #DemCast\nPresident Biden...         1             1
4  RT @CBSThisMorning: The Biden administration w...         1             1
3  @serena_patriot Expensive ass cult, ain't it? ...         0             1
2  RT @CBSNews: Marines tend to children in Afgha...         0             1
1  SUNDAY: Former U.S. Ambassador to the United N...         1             1
0  RT @charliespiering: At 1:00 pm Biden will del...         1             1


192.168.100.9 - - [20/Aug/2021 18:45:29] "POST /analyse HTTP/1.1" 200 -
192.168.100.9 - - [20/Aug/2021 18:45:29] "GET /static/images/cloud_plot.png HTTP/1.1" 200 -
192.168.100.9 - - [20/Aug/2021 18:45:29] "GET /static/images/new_plot.png HTTP/1.1" 200 -


                                                text  polarity  polarity_svm
9  RT @sanofi: We are excited to announce that Sa...         1             1
8  🇮🇳  #IndiaAt75 | On #IndependenceDay, we take ...         1             1
7  RT @sportstarweb: 🇮🇳  #IndiaAt75 | On #Indepen...         1             1
6  RT @sportstarweb: 🇮🇳  #IndiaAt75 | On #Indepen...         1             1
5  RT @48hours: Germany's coach for the modern pe...         1             1
4  RT @Tokyo2020: The #ClosingCeremony for the Ol...         1             1
3  RT @Tokyo_gov: Today, the Olympic Games #Tokyo...         1             1
2  @NULOOKREFINISH @legitgov 24,000 kids to be ja...         1             1
1  GameStop Deal of the Day [$37.99]: Mario and S...         1             1
0  RT @JoaquimCampa: Olympic Games 2024 https://t...         1             1


192.168.100.9 - - [20/Aug/2021 18:46:53] "POST /analyse HTTP/1.1" 200 -
192.168.100.9 - - [20/Aug/2021 18:46:53] "GET /static/images/cloud_plot.png HTTP/1.1" 200 -
192.168.100.9 - - [20/Aug/2021 18:46:53] "GET /static/images/new_plot.png HTTP/1.1" 200 -


                                                text  polarity  polarity_svm
9  A free shuttle will transport SAU students to ...         1             1
8  RT @CityLittleRock: COVID VACCINE CLINIC - Pfi...         1             1
7  RT @KenyaMedics_KMA: NEXT #TUESDAY\nYou are in...         1             1
6  RT @CityLittleRock: COVID VACCINE CLINIC - Pfi...         1             1
5  @ZankFrappa2 @ToadonaWire @ChristinaPushaw Pfi...         1             0
4  RT @CityLittleRock: COVID VACCINE CLINIC - Pfi...         1             1
3  RT @CityLittleRock: COVID VACCINE CLINIC - Pfi...         1             1
2  It's a #SuperSunday #Vaccination at @WatfordFC...         1             1
1  RT @Turmaine: It's a #SuperSunday #Vaccination...         1             1
0  RT @haskins_shelly: @maggieNYT Lot a lies to u...         1             1


192.168.100.9 - - [20/Aug/2021 18:58:06] "POST /analyse HTTP/1.1" 200 -
192.168.100.9 - - [20/Aug/2021 18:58:06] "GET /static/images/new_plot.png HTTP/1.1" 200 -
192.168.100.9 - - [20/Aug/2021 18:58:06] "GET /static/images/cloud_plot.png HTTP/1.1" 200 -
